In [1]:
# (\__/)    Still a little messy, but this code was written for the Mobility for All Abilities
# (•ㅅ•)     on 9/21/2019 as part of the 'Reliable Access to Subways' team organized by
# / 　 づ    TransitCenter and United Equal Access NY.

In [1]:
import PyPDF2 
from tqdm import tqdm_notebook as tqdm
import re
import pandas

In [3]:
def convert_transit_pdf_to_csv(pdf, start, end):
    
    '''
    Input is a PDF file in the local directory, the first page
    of the PDF and the last page of the PDF. The function adjusts
    the first page to account for zero-based numbering. It will
    output a csv file back into the directory.
    
    There are a few issues with the code as-written. If an escalator
    or elevator is avialable 0.00% of the time, it will add an additional
    digit or character to the availability column. There is one other
    issue I've encountered where the subway lines aren't formatted properly.
    
    The comments will occaisonaly cut off and a fix for that is the first
    priority once this code is re-visited.
    '''

    page_range = list(range(start - 1,end))

    pdfFileObj = open(pdf, 'rb') 

    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

    lines = []
    availability = []
    units = []
    stations = []
    conditions = []

    condition = ''

    for page in page_range:
        pageObj = pdfReader.getPage(page) 
        current_page = pageObj.extractText().split('\n')

        for i in range(len(current_page[:-2])):
            
            # removes some titles that would otherwise be caught up in 
            if not re.match(string = current_page[i], pattern = '.* THAN 85% AVAILABILITY'):
                if len(current_page[i]) > 1:
                    if len(current_page[i]) > 40:
                        condition += current_page[i]
                    if re.match(string = current_page[i][-5:], pattern = '\d\.\d{2}\%'):
                        availability.append(current_page[i][-6:])
                        lines.append(current_page[i][:-6])
                    if re.match(string = current_page[i], pattern = 'E[LS]\d{3}'):
                        units.append(current_page[i])
                        stations.append(current_page[i + 1])
                        if len(condition) > 1:
                            conditions.append(condition)
                            condition = ''
                if i == len(current_page[:-2]) - 1:
                    conditions.append(condition)
                    condition = ''
    df_stations = pd.DataFrame(
        {'units': units,
         'stations': stations,
         'lines' : lines,
         'availability' : availability,
         'condition': conditions
        })

    df_stations.to_csv(pdf + ' converted.csv')

In [419]:
convert_transit_pdf_to_csv('180521_1030_Transit.pdf', 220, 222)

ValueError: arrays must all be same length

In [399]:
df_stations

,units,stations,lines,availability,condition
0,EL120,190 St,A,27.18%,This elevator was out multiple times due to le...
1,EL249,86 St,Q,84.06%,This elevator was out of service from 11/19/18...
2,ES104,Gun Hill Rd,25,83.76%,From 10/1/18 to 12/23/18 the escalator went ou...
3,ES105,Gun Hill Rd,25,83.48%,From 10/1/18 to 12/23/18 the escalator went ou...
4,ES106,West Farms Sq-E Tremont Av,25,79.51%,From 10/1/18 to 12/23/18 the escalator went ou...
5,ES108,Intervale Av,25,69.84%,The escalator was out service from 10/17/2018 ...
6,ES114,161 St-Yankee Stadium,4,69.45%,The escalator was out of service from 10/18/20...
7,ES120,Pelham Bay Park,6,22.72%,The escalator was out of service from 09/01/20...
8,ES305,Court St,R,50.95%,This escalator was out of service from 10/6/18...
9,ES347,Broadway Junction,ACJLZ,84.90%,This escalator was out of service 13 times due...


In [420]:
len(conditions)

27

In [421]:
len(lines)

27

In [422]:
len(units)

27

In [423]:
len(stations)

27

In [424]:
len(availability)

27

In [425]:
current_page

['Borough/ Unit',
 'Location',
 '24 Hr ',
 'Availability',
 'Comments',
 '4TH QUARTER 2018 ESCALATORS WITH LESS THAN 85% AVAILABILITY',
 'ES235',
 '34 St-Herald Sq ',
 'BDFM84.34%',
 'The escalator was out of service from 12/4/18 to 12/11/18 due to worn out handrail and countershaft chains ',
 'as well as defective brake assemblies. The countershaft assembly and chain were replaced and adjusted. ',
 'The right handrail chain was adjusted. The main brakes were replaced and adjusted as well as a controller ',
 '',
 'relay; the escalator was tested and returned to service. The escalator was out multiple times due to the ',
 '',
 'activation of various safety devices. Those safety devices were tested and adjusted as needed.',
 'ES255',
 'Grand Central-42 St ',
 '4567S82.56%',
 'This escalator was out of service from 11/30/18 to 12/3/18 due to an inspection of the proper operation of ',
 'the up and down operation. The escalator was found to be operating as designed and returned to service.

In [4]:
def convert_transit_pdf_to_csv2(pdf, start, end):
    
    '''
    Input is a PDF file in the local directory, the first page
    of the PDF and the last page of the PDF. The function adjusts
    the first page to account for zero-based numbering. It will
    output a csv file back into the directory.
    
    There are a few issues with the code as-written. If an escalator
    or elevator is avialable 0.00% of the time, it will add an additional
    digit or character to the availability column. There is one other
    issue I've encountered where the subway lines aren't formatted properly.
    
    The comments will occaisonaly cut off and a fix for that is the first
    priority once this code is re-visited.
    '''

    page_range = list(range(start - 1,end))

    pdfFileObj = open(pdf, 'rb') 

    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

    lines = []
    availability = []
    units = []
    stations = []
    conditions = []

    condition = ''

    for page in page_range:
        pageObj = pdfReader.getPage(page) 
        current_page = pageObj.extractText().split('\n')

        for i in range(len(current_page[:-2])):
            
            # removes some titles that would otherwise be caught up in 
            if not re.match(string = current_page[i], pattern = '.* THAN 85% AVAILABILITY'):
                if len(current_page[i]) > 1:
                    if len(current_page[i]) > 40:
                        condition += current_page[i]
                    if re.match(string = current_page[i][-5:], pattern = '\d\.\d{2}\%'):
                        availability.append(current_page[i][-6:])
                        lines.append(current_page[i][:-6])
                    if re.match(string = current_page[i], pattern = 'E[LS]\d{3}'):
                        units.append(current_page[i])
                        stations.append(current_page[i + 1])
                        if len(condition) > 1:
                            conditions.append(condition)
                            condition = ''
                if i == len(current_page[:-2]) - 1:
                    conditions.append(condition)
                    condition = ''
    df_stations = pd.DataFrame(
        {'units': units,
         'stations': stations,
         'lines' : lines,
         'availability' : availability,
         'condition': conditions
        })

    df_stations.to_csv(pdf + ' converted.csv')

# A Tabula Approach

In [2]:
from tabula import read_pdf
import tabula

In [3]:
df = tabula.read_pdf('190923_1000_Transit.pdf', pages=[380, 381, 382, 383, 384])
df.dropna(how = 'all', inplace = True)
df.reset_index(inplace = True, drop = True)

In [272]:
df.head(4)

,2ND QUARTER 2019 ELEVATORS WITH LESS THAN 85% AVAILABILITY,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,Borough/ Unit,Location,24 Hr\rAvailability,Comments
1,BROOKLYN,NaN,NaN,NaN
2,EL204,Grand Central-42 St 4567S,72.36%,This elevator was out of service on multiple d...
3,EL119,181 St A,81.16%,This elevator was out of service from 6/3/19 t...


In [482]:
f = 0
unit = []
station = []
line = []
availability = []

for i in range(len(df)):
    try:
        if math.isnan(float(df.iloc[i][1])) is True:
            pass
    except:
        if df.iloc[i][1] != 'Location':
            unit.append(df.iloc[i][0])
            station.append(' '.join(df.iloc[i][1].split(' ')[:-1]))
            line.append(df.iloc[i][1].split(' ')[-1])
    if df.iloc[i][1] != 'Location':
        if df.iloc[i][2] != 'Location':
            if df.iloc[i].isna().sum() == 0:
                availability.append(df.iloc[i][2])
            if df.iloc[i].isna().sum() == 1:
                availability.append(df.iloc[i][3].split(' ')[0])
            
df_stations = pd.DataFrame(
    {'unit': unit,
     'station': station,
     'line' : line,
     'availability' : availability
    })

In [483]:
df_stations

,unit,station,line,availability
0,EL204,Grand Central-42 St,4567S,72.36%
1,EL119,181 St,A,81.16%
2,EL224,8 Av,L,83.42%
3,ES104,Gun Hill Rd,25,74.40%
4,ES105,Gun Hill Rd,25,79.55%
5,ES106,West Farms Sq-E Tremont Av,25,68.09%
6,ES108,Intervale Av,25,34.94%
7,ES112,Norwood-205 St,D,66.57%
8,ES113,161 St-Yankee Stadium,4,73.03%
9,ES114,161 St-Yankee Stadium,4,48.10%


In [161]:

type(df.iloc[1][1])

float

In [244]:
df

,2ND QUARTER 2019 ELEVATORS WITH LESS THAN 85% AVAILABILITY,Unnamed: 1,Unnamed: 2,Unnamed: 3
2,BROOKLYN,NaN,NaN,NaN
3,EL204,Grand Central-42 St 4567S,72.36%,This elevator was out of service on multiple d...
4,EL119,181 St A,81.16%,This elevator was out of service from 6/3/19 t...
5,EL224,8 Av L,83.42%,This elevator was out of service on multiple o...
6,NaN,NaN,NaN,2ND QUARTER 2019 ESCALATORS WITH LESS THAN 85%...
7,Borough/ Unit,NaN,Location,24 Hr Comments
8,NaN,NaN,NaN,Availability
9,BRONX,NaN,NaN,NaN
10,ES104,Gun Hill Rd 25,NaN,74.40% This escalator had multiple outages for...
11,NaN,NaN,NaN,device activations. The escalator was out of s...


In [14]:
import math

In [214]:
math.isnan(float(df.iloc[0][1]))

True

In [334]:
for i in range(len(df)):
    try:
        if math.isnan(float(df.iloc[i][1])) is True:
            pass
    except:
        if re.match(string = df.iloc[i][-1].split(' ')[0], pattern = '\d+\.\d{2}\%'):
            print(' '.join(df.iloc[i][-1].split(' ')[1:]))

This escalator had multiple outages for lower combstop safety device activations and step-up thrust safety
This escalator was out of service multiple times for various activations of the escalator safety devices. The
This escalator was out of service multiple times due to vandalism. The emergency stop buttons were being
This escalator was out of service from 4/23/19 to 6/14/19 due to a step wreck. Various step chains and step
This escalator was out of service from 4/19/19 to 5/17/19 due to a safety check and related work. The comb
This escalator had repeated outages for combstop and comb impact safety activations. The escalator was
This escalator was out of service from 4/11/19 to 5/17/19 due to a safety check and related repair work.
This escalator was out of service from 4/16/19 to 4/23/19 for a safety check and related work. The skirts,
This escalator had various multiple outages for phase reversal, combstop safety devices activating and
This escalator was out of service from 4/13/1

In [356]:
df[df.columns[-1]]

0                                               Comments
1                                                    NaN
2      This elevator was out of service on multiple d...
3      This elevator was out of service from 6/3/19 t...
4      This elevator was out of service on multiple o...
5      2ND QUARTER 2019 ESCALATORS WITH LESS THAN 85%...
6                                         24 Hr Comments
7                                           Availability
8                                                    NaN
9      74.40% This escalator had multiple outages for...
10     device activations. The escalator was out of s...
11     correct repeated outages for the lower combsto...
12     were adjusted; the escalator was tested and re...
13     79.55% This escalator was out of service multi...
14     upper and lower emergency stop buttons were ac...
15     combstop safety devices were activated. Each t...
16     service. Further work is required to inspect, ...
17     combstop safety devices 

In [384]:
comment = ''

In [488]:
comments = []
comment = ''

raw_comments = df[df.columns[-1]]

for i in range(len(raw_comments)):
    try:
        if math.isnan(float(raw_comments[i])) is True:
            pass
    except:
        if len(raw_comments[i]) > 12:
            if re.match(string = raw_comments[i], pattern = '\d+\.\d{2}\%'):
                comments.append(comment)
                comment = ''
                comment += ' '.join(raw_comments[i].split(' ')[1:])
            if re.match(string = raw_comments[i], pattern = '^[Th]'):
                comments.append(raw_comments[i])
            if re.match(string = raw_comments[i], pattern = '^[a-zA-Z]'):
                comment += raw_comments[i]
                
comments = list(filter(None, comments))
                
for j in range(len(comments)):
    comments[j] = comments[j].replace("\r", " ")

In [491]:
len(comments)

64

In [490]:
comments

['This elevator was out of service on multiple days in April due to planned maintenance repairs. The elevator was out of service from 5/6/19 to 5/8/19 due to the hydraulic jack leaking. The hydraulic jack seals were replaced and the piston was filed; the elevator was tested and returned to service. The elevator was out of service from 5/13/19 to 5/15/19 due to an entrapment investigation. Repairs and adjustments were made to the car and hoistway doors and interlocks, rail guide clamps in the shaftway were secured and the elevator cab was squared and secured; the elevator was tested and returned to service. The elevator is scheduled for replacement in the 2015-2019 Capital Replacement Program; the replacement work is expected to start later this year.',
 'This elevator was out of service from 6/3/19 to 6/13/19 due to defective elevator brakes. The elevator brakes were repaired; the elevator was tested and returned to service. he elevator is scheduled for replacement in the 2015-2019 Cap

In [470]:
len(set(comments)) - len(comments)

0

In [472]:
len(comments)

63

In [463]:
import pandas as pd

In [464]:
pd.options.display.max_rows = 4000

In [465]:
raw_comments

0                                               Comments
1                                                    NaN
2      This elevator was out of service on multiple d...
3      This elevator was out of service from 6/3/19 t...
4      This elevator was out of service on multiple o...
5      2ND QUARTER 2019 ESCALATORS WITH LESS THAN 85%...
6                                         24 Hr Comments
7                                           Availability
8                                                    NaN
9      74.40% This escalator had multiple outages for...
10     device activations. The escalator was out of s...
11     correct repeated outages for the lower combsto...
12     were adjusted; the escalator was tested and re...
13     79.55% This escalator was out of service multi...
14     upper and lower emergency stop buttons were ac...
15     combstop safety devices were activated. Each t...
16     service. Further work is required to inspect, ...
17     combstop safety devices 

In [411]:
q = 0
for i in range(len(comments)):
    if len(comments[i]) == 0:
        q += 1
q

1

In [360]:
print(re.match(string = raw_comments[10], pattern = '^[a-z]'))

<re.Match object; span=(0, 1), match='d'>


'device'

'nan'

In [131]:
df.iloc[i][1].split(' ')[-1]

AttributeError: 'float' object has no attribute 'split'

no


4

In [4]:
comments = []
comment = ''

raw_comments = df[df.columns[-1]]

In [64]:
raw_comments

0                                               Comments
1                                                    NaN
2      This elevator was out of service on multiple d...
3      This elevator was out of service from 6/3/19 t...
4      This elevator was out of service on multiple o...
5      2ND QUARTER 2019 ESCALATORS WITH LESS THAN 85%...
6                                         24 Hr Comments
7                                           Availability
8                                                    NaN
9      74.40% This escalator had multiple outages for...
10     device activations. The escalator was out of s...
11     correct repeated outages for the lower combsto...
12     were adjusted; the escalator was tested and re...
13     79.55% This escalator was out of service multi...
14     upper and lower emergency stop buttons were ac...
15     combstop safety devices were activated. Each t...
16     service. Further work is required to inspect, ...
17     combstop safety devices 

In [63]:
thread = 0
for i in range(len(raw_comments)):
    try:
        if math.isnan(float(raw_comments[i])) is True:
            pass
    except:
        if re.match(string = raw_comments[i], pattern = '^Th.*$'):
            print(raw_comments[i])

Replacement Program; the replacement work is expected to start later this year.
expected to start later this year.
and adjusted; the elevator was inspected, tested and returned to service.
escalator was tested and returned to service.
was tested and returned to service.
replaced, the steps and axles are being replaced. The escalator currently remains out of service.
replaced; the escalator was tested and returned to service.
planned to mitigate these repeated failures and improve the reliability and availability of this escalator.
order to mitigate the repeated failures and improve the escalator reliability and availability.
The brake coil and sensors were replaced and adjusted; the escalator was tested and returned to service.
repaired; the escalator was tested and returned to service.
and returned to service.
was tested and returned to service.
and returned to service.
combstop and impact safety device were adjusted; the escalator was tested and returned to service.
was replaced and 

In [52]:
raw_comments[i]

'The escalator was out of service from 4/22/19 to 7/2/19 due to extensive escalator repairs. The gearcase\rwere removed and replaced; the escalator was tested and returned to service.'